In [1]:

# 1. pytest 
# 2. cache
# 3. pydantic object and output parser
# 4. token counter(cost analysis)
# 5. history or memory 
# 6. evaluation
# 7. guadrails
# 8. adavance RAG

## as a experiment will do in the notebook
### then as a assignment you can integrate it in end to end project

## Next Saturday there will demonstration of this concept (whoever will comeplete it
# that person will give the demo)

### next week satuday 2-2:30

### then will start with the next project

#For first winner there is prize money:

Assignment on first Project:
1. you have to make enable this project for every document(.ppt,.docx,.md,.txt,.pdf,.xlxs,.csv,anysqldb)
2. you have to add a code for dealing with table and images data also
3. you have to add the evalation matrix using the DeepEval
4. write at least 10 test cases
5. this cases should be vaildate before the commit and after the commit
6. add one login screen to your protal
7. use the langchain inmemory cache and implement inside the project
8. deadline for this assignment is till 5th of september(friday)
9. inner will present the entire solution to the whole class (30 minute presentation would be there)
10. prize: money or course or giftpack(there will be only one winnner)

In [1]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from utils.model_loader import ModelLoader

In [2]:
loader = ModelLoader()

{"timestamp": "2025-10-22T01:24:18.929345Z", "level": "info", "event": "Running in LOCAL mode: .env loaded"}
{"timestamp": "2025-10-22T01:24:18.930023Z", "level": "info", "event": "Loaded GROQ_API_KEY from individual env var"}
{"timestamp": "2025-10-22T01:24:18.931111Z", "level": "info", "event": "Loaded GOOGLE_API_KEY from individual env var"}
{"keys": {"GROQ_API_KEY": "gsk_Dm...", "GOOGLE_API_KEY": "AIzaSy..."}, "timestamp": "2025-10-22T01:24:18.932029Z", "level": "info", "event": "API keys loaded"}
{"config_keys": ["faiss_db", "embedding_model", "retriever", "llm"], "timestamp": "2025-10-22T01:24:18.935342Z", "level": "info", "event": "YAML config loaded"}


In [3]:
llm = loader.load_llm()
print(f"LLM Loaded: {llm}")
result = llm.invoke("Hello, how are you?")
print(f"LLM Result: {result.content}")


{"provider": "google", "model": "gemini-2.0-flash", "timestamp": "2025-10-22T01:24:21.537056Z", "level": "info", "event": "Loading LLM"}


LLM Loaded: model='models/gemini-2.0-flash' google_api_key=SecretStr('**********') temperature=0.0 max_output_tokens=2048 client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x107c347d0> default_metadata=() model_kwargs={}
LLM Result: I am doing well, thank you for asking! How are you today?


In [4]:
parser = JsonOutputParser()

In [5]:
parser.get_format_instructions()

'Return a JSON object.'

In [6]:
prompt = PromptTemplate.from_template(
    "Give me a JSON with keys 'title' and 'summary' for this topic: {topic}\n{format_instructions}"
).partial(format_instructions=parser.get_format_instructions())


In [7]:
prompt

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={'format_instructions': 'Return a JSON object.'}, template="Give me a JSON with keys 'title' and 'summary' for this topic: {topic}\n{format_instructions}")

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={'format_instructions': 'Return a JSON object.'}, template="Give me a JSON with keys 'title' and 'summary' for this topic: {topic}\n{format_instructions}")

In [8]:
chain = prompt | llm | parser

In [9]:
result = chain.invoke({"topic": "LangChain for RAG"})

In [10]:
result

{'title': 'LangChain for Retrieval Augmented Generation (RAG)',
 'summary': 'LangChain simplifies the development of Retrieval Augmented Generation (RAG) applications. It provides modules for connecting to data sources, creating embeddings, indexing data, and building chains that combine retrieval and generation models. This allows developers to build powerful applications that leverage external knowledge to enhance the quality and relevance of generated text.'}

In [12]:
from langchain.output_parsers import OutputFixingParser

In [13]:
json_parser = JsonOutputParser()

In [14]:
# Fixing wrapper
fixing_parser = OutputFixingParser.from_llm(parser=json_parser, llm=llm)

In [15]:
# Broken JSON
bad_output = """
title: LangChain RAG
summary LangChain helps with retrieval augmented generation...
"""

In [16]:

fixed_result = fixing_parser.parse(bad_output)

In [17]:
print(fixed_result)

{'title': 'LangChain RAG', 'summary': 'LangChain helps with retrieval augmented generation...'}


| Feature                | `JsonOutputParser`               | `OutputFixingParser`                     
| ---------------------- | -------------------------------- | ---------------------------------------- 
| Purpose                | Enforce strict JSON              | Auto-correct bad/invalid output          
| Works with             | Well-formed model responses only | Even with malformed model responses      
| Raises error if broken | Yes                            |   No, it tries to fix using LLM          
| Ideal for              | Strict structured output         | Fragile prompts, semi-structured outputs 



In [ ]:
# from langchain.output_parsers import OutputFixingParser
# from langchain_core.output_parsers import JsonOutputParser

# json_parser = JsonOutputParser()
# safe_parser = OutputFixingParser.from_llm(parser=json_parser, llm=llm)

# # Use safe_parser instead of json_parser
# chain = prompt | llm | safe_parser


In [18]:
# Wrap with cache
Model_Cache = {}

import time
def cached_model(query):
    start_time = time.time()
    if Model_Cache.get(query):
        print("***CACHE HIT***")
        end_time = time.time()
        elapsed = end_time - start_time
        print(f"EXECUTION TIME: {elapsed:.2f} seconds")
        return Model_Cache.get(query)
    else:
        print("***CACHE MISS – EXECUTING MODEL***")
        start_time = time.time()
        response = llm.invoke(query)
        end_time = time.time()
        elapsed = end_time - start_time
        print(f"EXECUTION TIME: {elapsed:.2f} seconds")
        Model_Cache[query] = response
        return response

In [19]:
query="hi"
response = cached_model(query)
print(response)

***CACHE MISS – EXECUTING MODEL***
EXECUTION TIME: 0.56 seconds
content='Hi there! How can I help you today?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--9a568ffa-754a-4d67-b034-2c06828f4919-0' usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}}


In [20]:
query="can you give me 1000 words essay on independence?"
response = cached_model(query)
print(response)

***CACHE MISS – EXECUTING MODEL***
EXECUTION TIME: 8.94 seconds
content="## The Enduring Quest for Independence: A Multifaceted Exploration\n\nIndependence, a word that resonates with power and promise, is a concept deeply woven into the fabric of human existence. It signifies the state of being free from the control, influence, or support of others, allowing for self-governance, autonomy, and the pursuit of one's own path. While often associated with political liberation and national sovereignty, independence extends far beyond the realm of nations, encompassing personal, economic, intellectual, and even spiritual dimensions. This essay will explore the multifaceted nature of independence, examining its historical significance, its psychological underpinnings, its economic implications, and the challenges and responsibilities that accompany its attainment.\n\nHistorically, the pursuit of independence has been a driving force behind countless revolutions and social movements. From the 

In [21]:
query="can you give me 1000 words essay on independence?"
response = cached_model(query)
print(response)

***CACHE HIT***
EXECUTION TIME: 0.00 seconds
content="## The Enduring Quest for Independence: A Multifaceted Exploration\n\nIndependence, a word that resonates with power and promise, is a concept deeply woven into the fabric of human existence. It signifies the state of being free from the control, influence, or support of others, allowing for self-governance, autonomy, and the pursuit of one's own path. While often associated with political liberation and national sovereignty, independence extends far beyond the realm of nations, encompassing personal, economic, intellectual, and even spiritual dimensions. This essay will explore the multifaceted nature of independence, examining its historical significance, its psychological underpinnings, its economic implications, and the challenges and responsibilities that accompany its attainment.\n\nHistorically, the pursuit of independence has been a driving force behind countless revolutions and social movements. From the American Revolution

In [22]:
embedding_model = loader.load_embeddings()
print(f"Embedding Model Loaded: {embedding_model}")
result = embedding_model.embed_query("Hello, how are you?")
print(f"Embedding Result: {result}")


{"model": "models/text-embedding-004", "timestamp": "2025-10-22T01:25:29.919964Z", "level": "info", "event": "Loading embedding model"}


Embedding Model Loaded: client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x111224590> async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x1112203b0> model='models/text-embedding-004' task_type=None google_api_key=SecretStr('**********') credentials=None client_options=None transport=None request_options=None
Embedding Result: [0.012455824762582779, -0.022220758721232414, -0.050555989146232605, -0.04617856815457344, 0.007134224288165569, 0.03131191432476044, 0.03413437679409981, 0.002971651265397668, -0.010995007120072842, 0.04237418621778488, -0.01806010864675045, 0.02493678405880928, 0.1121494323015213, -0.013102629221975803, -0.0148635134100914, -0.024023527279496193, 0.010264375247061253, -0.0037090936675667763, -0.10687677562236786, -0.001329770078882575, 0.03868139907717705, -0.03769991546869278, 0.03614252060651779, 0.005920442286878824, 

In [23]:
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableMap, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [24]:
from langchain.schema import Document

documents = [
    Document(page_content="The Earth is the third planet from the Sun and the only known planet to support life. It has a diverse climate, ranging from arctic to tropical zones, and supports ecosystems across seven continents and five oceans."),

    Document(page_content="The Industrial Revolution, beginning in the 18th century, drastically transformed human societies by shifting from manual labor to machine-based manufacturing, leading to urbanization and economic expansion globally."),

    Document(page_content="The United Nations, established in 1945 after World War II, is an international organization founded to promote peace, security, human rights, and cooperation among countries. It has 193 member states."),

    Document(page_content="The global economy is an interconnected system involving trade, investment, and financial flows across countries. Major players include the United States, China, the European Union, and emerging markets like India and Brazil."),

    Document(page_content="Climate change refers to long-term shifts in temperatures and weather patterns. It is largely driven by human activities like burning fossil fuels, deforestation, and industrial emissions, leading to global warming and sea level rise."),

    Document(page_content="Democracy is a political system in which citizens exercise power by voting. Modern democracies typically have institutions for free elections, rule of law, freedom of expression, and checks and balances."),

    Document(page_content="The Internet has revolutionized communication, commerce, and education worldwide. Originating from military research in the 1960s, it now connects over 5 billion people, enabling instant global information exchange."),

    Document(page_content="Renewable energy sources like solar, wind, hydro, and geothermal are critical for a sustainable future. They offer alternatives to fossil fuels, reducing carbon emissions and reliance on finite resources."),

    Document(page_content="The World Health Organization (WHO) is a UN agency focused on global health issues. It coordinates international efforts to monitor diseases, set health standards, and respond to pandemics like COVID-19."),

    Document(page_content="Globalization is the process of increasing interaction and integration among people, companies, and governments worldwide. It has led to greater economic growth but also raised concerns about inequality and cultural homogenization.")
]

In [26]:
# Chroma vector DB with persistent storage
vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory="./chroma_db"  # Disk path for persistence
)

Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [27]:
# Optional: Persist manually (though auto-persistence happens internally)
vector_store.persist()

/var/folders/rv/z_0yz9v11p94xxdj7r_1w4vc0000gn/T/ipykernel_28942/1854114808.py:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


In [28]:
retriever = vector_store.as_retriever()

In [29]:
prompt = PromptTemplate.from_template(
    """
    Use the following context to answer the question.
    If you don't know the answer, just say you don't know. Don't try to make up an answer.

    Context:
    {context}

    Question:
    {question}

    Answer:
    """
)

In [30]:
# LCEL RAG Chain step-by-step
rag_chain = (
    RunnableMap({
        "context": retriever | (lambda docs: "\n\n".join([doc.page_content for doc in docs])),
        "question": RunnablePassthrough()
    })
    | prompt
    | llm
    | StrOutputParser()
)

In [31]:
# Wrap with cache
RAG_Cache = {}

In [32]:
import time
def cached_rag_chain(query):
    start_time = time.time()
    if RAG_Cache.get(query):
        print("***CACHE HIT***")
        end_time = time.time()
        elapsed = end_time - start_time
        print(f"EXECUTION TIME: {elapsed:.2f} seconds")
        return RAG_Cache.get(query)
    else:
        print("***CACHE MISS – EXECUTING MODEL***")
        start_time = time.time()
        response = llm.invoke(query)
        end_time = time.time()
        elapsed = end_time - start_time
        print(f"EXECUTION TIME: {elapsed:.2f} seconds")
        RAG_Cache[query] = response
        return response

In [33]:
query = "what is japan economy in 2024 and relation with north korea?"
response = cached_rag_chain(query)
print(response)

***CACHE MISS – EXECUTING MODEL***
EXECUTION TIME: 9.82 seconds
content="Okay, let's break down the Japanese economy in 2024 and its relationship with North Korea.\n\n**Japan's Economy in 2024: Overview**\n\nAs of late 2024, Japan's economy is navigating a complex landscape. Here's a summary of key aspects:\n\n*   **Growth:** Japan's economic growth is expected to be modest. The IMF and other organizations project growth in the range of 0.5-1% for 2024. This is influenced by factors like global economic conditions, domestic demand, and government policies.\n*   **Inflation:** Japan has been experiencing inflation, a departure from its long history of deflation. The Bank of Japan (BOJ) has been closely monitoring this, and there have been some adjustments to monetary policy, including ending negative interest rates. The goal is to achieve sustainable inflation driven by wage growth and demand, rather than just cost-push factors.\n*   **Monetary Policy:** The Bank of Japan (BOJ) has hist

In [34]:
query = "what is langchain framework?"
response = cached_rag_chain(query)
print(response)

***CACHE MISS – EXECUTING MODEL***
EXECUTION TIME: 6.74 seconds
content="LangChain is a framework designed to simplify the development of applications powered by large language models (LLMs). It provides tools, components, and interfaces to connect LLMs to various data sources and other computational resources, enabling developers to build more complex and sophisticated AI applications.\n\nHere's a breakdown of what makes LangChain useful:\n\n**Key Features and Components:**\n\n*   **LLM Integration:** LangChain provides a standardized interface for interacting with various LLMs, including OpenAI's GPT models, Cohere, Hugging Face models, and more. This allows you to easily switch between models or experiment with different LLMs without significantly altering your code.\n\n*   **Data Connection:**  A core strength of LangChain is its ability to connect LLMs to external data sources. This allows you to ground the LLM's responses in real-world information, making them more accurate and r

In [35]:
query = "Why United Nations, established in 1945 after World War II?"
response = cached_rag_chain(query)
print(response)

***CACHE MISS – EXECUTING MODEL***
EXECUTION TIME: 4.53 seconds
content="The United Nations was established in 1945 after World War II for a multitude of crucial reasons, all stemming from the desire to prevent another global conflict and foster international cooperation. Here's a breakdown of the key motivations:\n\n*   **Preventing Future Wars:** The primary and overarching goal was to prevent another devastating world war like World War II. The war had caused unprecedented death, destruction, and suffering. The UN was designed to provide a forum for nations to discuss their differences, resolve disputes peacefully, and collectively address threats to international peace and security.\n\n*   **Replacing the League of Nations:** The League of Nations, established after World War I, had failed to prevent World War II. It was seen as weak and ineffective, lacking the necessary authority and support from major powers. The UN was intended to be a stronger, more effective organization with

## Cache Using LangChain

In [36]:
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache
from typing import Any, Dict, Tuple

In [37]:
class DebuggableCache(InMemoryCache):
    def __init__(self):
        super().__init__()
        self._cache: Dict[Tuple[str, str], Any] = {}

    def lookup(self, prompt: str, llm_string: str):
        return self._cache.get((prompt, llm_string))

    def update(self, prompt: str, llm_string: str, return_val: Any):
        self._cache[(prompt, llm_string)] = return_val

    def view_cache(self):  # this is our custom method
        return self._cache

In [38]:
dbg_cache = DebuggableCache()
set_llm_cache(dbg_cache)

In [39]:
response = llm.invoke("What is the capital of France?")

In [40]:
print("LLM Response:", response)

LLM Response: content='The capital of France is **Paris**.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--559d5eb4-8d45-467e-8f8f-2250eae8dd87-0' usage_metadata={'input_tokens': 7, 'output_tokens': 9, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}}


In [41]:
print("\nCache Contents:")
for k, v in dbg_cache.view_cache().items():
    print(f"Prompt: {k[0]} | Cached Output: {v}")


Cache Contents:
Prompt: [{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "HumanMessage"], "kwargs": {"content": "What is the capital of France?", "type": "human"}}] | Cached Output: [ChatGeneration(text='The capital of France is **Paris**.', generation_info={'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, message=AIMessage(content='The capital of France is **Paris**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--559d5eb4-8d45-467e-8f8f-2250eae8dd87-0', usage_metadata={'input_tokens': 7, 'output_tokens': 9, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}}))]


In [42]:
response = llm.invoke("What is the capital of France?")

In [43]:
print("\nCache Contents:")
for k, v in dbg_cache.view_cache().items():
    print(f"Prompt: {k[0]} | Cached Output: {v}")


Cache Contents:
Prompt: [{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "HumanMessage"], "kwargs": {"content": "What is the capital of France?", "type": "human"}}] | Cached Output: [ChatGeneration(text='The capital of France is **Paris**.', generation_info={'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, message=AIMessage(content='The capital of France is **Paris**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--559d5eb4-8d45-467e-8f8f-2250eae8dd87-0', usage_metadata={'input_tokens': 7, 'output_tokens': 9, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}}))]


In [44]:
response = llm.invoke("What is the capital of india?")
print("LLM Response:", response)

LLM Response: content='The capital of India is **New Delhi**.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--dd0e13ca-edc6-447f-8133-3eea989c91e2-0' usage_metadata={'input_tokens': 7, 'output_tokens': 10, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}}


In [45]:
print("\nCache Contents:")
for k, v in dbg_cache.view_cache().items():
    print(f"Prompt: {k[0]} | Cached Output: {v}")


Cache Contents:
Prompt: [{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "HumanMessage"], "kwargs": {"content": "What is the capital of France?", "type": "human"}}] | Cached Output: [ChatGeneration(text='The capital of France is **Paris**.', generation_info={'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, message=AIMessage(content='The capital of France is **Paris**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--559d5eb4-8d45-467e-8f8f-2250eae8dd87-0', usage_metadata={'input_tokens': 7, 'output_tokens': 9, 'total_tokens': 16, 'input_token_details': {'cache_read': 0}}))]
Prompt: [{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "HumanMessage"], "kwargs": {"content": "What is the capital of india?", "type": "human"}}] | Cached Output: [ChatGeneration(tex

In [46]:
response = llm.invoke("What is the capital of india?")
print("LLM Response:", response)

LLM Response: content='The capital of India is **New Delhi**.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--dd0e13ca-edc6-447f-8133-3eea989c91e2-0' usage_metadata={'input_tokens': 7, 'output_tokens': 10, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}}


In [47]:
response = llm.invoke("give me 1000 lines of essay on science and give the importance of it regarding mathematics?")
print("LLM Response:", response)

LLM Response: content="## The Indelible Bond: Science, Mathematics, and the Pursuit of Understanding\n\nScience, in its broadest sense, is the systematic pursuit of knowledge about the natural world through observation, experimentation, and analysis. It is a dynamic and ever-evolving process, driven by curiosity and a desire to understand the intricate workings of the universe. From the smallest subatomic particles to the vast expanse of galaxies, science seeks to unravel the mysteries that surround us, providing explanations and predictions that shape our understanding of reality. At the heart of this endeavor lies a crucial and indispensable tool: mathematics. Mathematics provides the language, the structure, and the logical framework upon which scientific theories are built, tested, and refined. The relationship between science and mathematics is not merely one of utility; it is a symbiotic partnership, a fundamental and inseparable bond that fuels scientific progress and deepens ou

In [48]:
from langchain.callbacks import get_openai_callback
with get_openai_callback() as cb:
    response = llm.invoke("Tell me a joke about LangChain and ECS")
    print("Response:", response.content)
    print("Token Usage Stats:", cb)

Response: Okay, here's a joke about LangChain and ECS:

Why did the LangChain agent refuse to deploy on ECS?

Because it kept saying, "I can't find the context! I need more environment variables!  I'm pretty sure my memory is leaking, and I'm not even sure if I'm running in a container or just hallucinating the whole thing!"

...Turns out, the ECS task definition was missing the necessary IAM role to access the S3 bucket where the agent's knowledge base was stored.  It was a classic case of "no context, no content!"
Token Usage Stats: Tokens Used: 136
	Prompt Tokens: 9
		Prompt Tokens Cached: 0
	Completion Tokens: 127
		Reasoning Tokens: 0
Successful Requests: 1
Total Cost (USD): $0.0


In [49]:
questions = [
    "What is Retrieval-Augmented Generation?",
    "How does FAISS indexing work?",
    "What is the difference between fine-tuning and RAG?"
]

with get_openai_callback() as cb:
    for q in questions:
        answer = llm.invoke(q)
        print(f"\nQ: {q}\nA: {answer.content}")

    print("\n=== Token Usage Summary ===")
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost:.6f}")



Q: What is Retrieval-Augmented Generation?
A: Retrieval-Augmented Generation (RAG) is a technique that enhances the capabilities of large language models (LLMs) by allowing them to access and incorporate information from external knowledge sources during the generation process.  Think of it as giving an LLM a textbook or a library to consult before answering a question.

Here's a breakdown of the key components and how it works:

**1. Retrieval:**

*   **Knowledge Source:** This is the external database or repository containing the information the LLM will use. It could be:
    *   A collection of documents (PDFs, text files, web pages)
    *   A knowledge graph
    *   A database
    *   A vector database (more on this below)
*   **Query Generation:** When a user asks a question or provides a prompt, the RAG system generates a query to search the knowledge source. This query is often a vector representation of the user's input, designed to capture the semantic meaning.
*   **Informat